In [1]:
import pandas as pd
pd.set_option('display.max_columns',500)

In [2]:
user_data = pd.read_csv("user.csv")
content_data = pd.read_csv("content.csv")
relation_data = pd.read_csv("relationship.csv")

In [3]:
test_data = pd.read_csv("test.csv")

# Content Based approach:

>#### Step 1: Created content-content similarity using metadata *using the similarity score as Cosine*
>#### Step 2: Given test file has 2 kinds of users 1. Existing users, 2. Cold start users(new users).
>#### Step 2.1: For Existing users taking the user already watched content and push recommendations similar to the watched content based on the frequency of content suggested
>#### Step 2.2: For Cold Start users finding the similar users based onthe users metadata and pushing the recommendations based on the content rating and frequency of the content suggested

## Alternative approaches experimented:
>#### 1. As there is no user-content rating explicitly given
>#### 2. After trying collaborative filtering and using embedding
>#### 3. Tried the approach of collaborative filtering of certain presumptions outputs are biased towards general content ratings.


In [4]:
content_data['content_duraiton_mins'] = content_data['duration']/60000

In [5]:
content_data['content_val'] = content_data['content_id'].apply(lambda x: x.split("_")[1])

In [6]:
# content_data

In [7]:
import time,datetime

In [8]:
content_data['content_yrs'] = content_data['release_date'].apply(
    lambda x: round((datetime.datetime.now()-pd.to_datetime(x)).total_seconds()/(365.25*24*60*60),2))

In [9]:
# from sklearn.preprocessing import StandardScaler
# # scaler = StandardScaler()

# from sklearn.preprocessing import MinMaxScaler
# minmaxsclar = MinMaxScaler()
#'total_episodes', 'content_duraiton_mins', 'content_yrs','rating'
# inverse = scaler.inverse_transform(normalized)
# content_data[[ 'content_duraiton_mins_scl', 'content_yrs_scl','rating_scl','episode_count_scl',
#               'season_count_scl']]=minmaxsclar.fit_transform(
#     content_data[['content_duraiton_mins', 'content_yrs','rating','episode_count','season_count']])

In [10]:
user_data['user_yrs'] = user_data['joining_date'].apply(
    lambda x: round((datetime.datetime.now()-pd.to_datetime(x)).total_seconds()/(365.25*24*60*60),2))

In [11]:
# User meta data mappings
# user---> 'user_age', 'gender' ,'location' 'user_yrs'

In [12]:
relation_data.sort_values(['date'],ascending=False,inplace=True)

In [13]:
relation_data['user_watch_dur_mins'] = relation_data['duration']/60000
relation_grpby = relation_data.groupby(['user_id','content_id']).agg({'user_watch_dur_mins':'sum','date':'first'}).reset_index()
# relation_grpby['user_watch_dur_mins_scl'] = minmaxsclar.fit_transform(relation_grpby['user_watch_dur_mins'].values.reshape(-1,1))

In [14]:
# popularity_df = relation_grpby.groupby(['content_id'])['user_id'].count().reset_index().sort_values(['user_id'],ascending=False)

In [15]:
# popularity_content_dict={}
# for i,row in popularity_df.iterrows():
#     popularity_content_dict[row['content_id']]=row['user_id']

In [16]:
user_data['user_age_bin'] = pd.cut(user_data['user_age'], [0,20,30,40,50,100], labels=[0,1,2,3,4])
user_data['user_age_bin'] = user_data['user_age_bin'].astype(str)

In [17]:
user_data.columns

Index(['user_id', 'user_age', 'gender', 'location', 'joining_date', 'user_yrs',
       'user_age_bin'],
      dtype='object')

In [18]:
user_dummies=pd.concat([user_data[['user_id','user_yrs']],pd.get_dummies(user_data[['gender','location','user_age_bin']])],axis=1)

In [19]:
content_data['content_val'] = content_data['content_val'].apply(str)
content_data['season_count'] = content_data['season_count'].apply(str)

In [20]:
# content_data['content_yrs'].hist()

content_data['content_yrs'] = pd.cut(content_data['content_yrs'], [0,2,5,100], labels=[0,1,2])
content_data['content_yrs'] = content_data['content_yrs'].astype(str)

In [21]:
# content_data['popularity_rank']=content_data['content_id'].map(popularity_content_dict)

In [22]:
# content_data['popularity_rank'].fillna(0,inplace=True)

In [23]:
content_dummies=pd.concat([content_data[['content_id','episode_count', 'content_duraiton_mins',
       'rating']],pd.get_dummies(content_data[['content_type', 'language', 'genre','content_val',
                                               'season_count','content_yrs']])],axis=1)

In [24]:
# content_dummies.to_pickle("content_Dummies_pkl.pkl")

In [25]:
content_dummies.head(3)

,content_id,episode_count,content_duraiton_mins,rating,content_type_movies,content_type_series,content_type_sports,content_type_teasers,language_bengali,language_english,language_gujarati,language_hindi,language_kannada,language_malayalam,language_marathi,language_oriya,language_punjabi,language_tamil,language_telugu,genre_action,genre_adventure,genre_animation,genre_badminton,genre_basketball,genre_biography,genre_comedy,genre_cricket,genre_crime,genre_documentary,genre_drama,genre_family,genre_fantasy,genre_football,genre_hockey,genre_horror,genre_musical,genre_mystery,genre_sci-fi,genre_sport,genre_tennis,genre_thriller,content_val_1,content_val_10,content_val_100,content_val_1000,content_val_1002,content_val_1003,content_val_1005,content_val_1006,content_val_1007,content_val_1008,content_val_1009,content_val_101,content_val_1010,content_val_1012,content_val_1013,content_val_1015,content_val_1017,content_val_102,content_val_1020,content_val_1021,content_val_1022,content_val_1023,content_val_1025,content_val_1026,content_val_1027,content_val_1028,content_val_103,content_val_1030,content_val_1031,content_val_1032,content_val_1034,content_val_1037,content_val_1038,content_val_104,content_val_1040,content_val_1042,content_val_1043,content_val_1044,content_val_1045,content_val_1046,content_val_1047,content_val_1050,content_val_1051,content_val_1053,content_val_1055,content_val_1056,content_val_1057,content_val_1059,content_val_1060,content_val_1061,content_val_1062,content_val_1063,content_val_1065,content_val_1067,content_val_1068,content_val_1069,content_val_107,content_val_1072,content_val_1075,content_val_1076,content_val_1077,content_val_1078,content_val_1081,content_val_1082,content_val_1083,content_val_1085,content_val_1087,content_val_1088,content_val_1089,content_val_1090,content_val_1091,content_val_1092,content_val_1093,content_val_1094,content_val_1096,content_val_1097,content_val_1098,content_val_1099,content_val_11,content_val_110,content_val_1100,content_val_1101,content_val_1102,content_val_1103,content_val_1104,content_val_1106,content_val_1108,content_val_1115,content_val_1116,content_val_1118,content_val_1119,content_val_112,content_val_1120,content_val_1121,content_val_1122,content_val_1123,content_val_1126,content_val_1128,content_val_113,content_val_1130,content_val_1135,content_val_1136,content_val_1137,content_val_1139,content_val_114,content_val_1140,content_val_1141,content_val_1142,content_val_1143,content_val_1148,content_val_1149,content_val_1150,content_val_1151,content_val_1154,content_val_1155,content_val_1156,content_val_1157,content_val_1158,content_val_116,content_val_1163,content_val_1164,content_val_1166,content_val_1167,content_val_1168,content_val_1169,content_val_117,content_val_1170,content_val_1171,content_val_1172,content_val_1175,content_val_1177,content_val_1178,content_val_1179,content_val_1181,content_val_1183,content_val_1185,content_val_1186,content_val_1187,content_val_1190,content_val_1191,content_val_1192,content_val_1193,content_val_1195,content_val_1197,content_val_1198,content_val_1199,content_val_12,content_val_120,content_val_1200,content_val_1202,content_val_1203,content_val_1205,content_val_1206,content_val_1207,content_val_1208,content_val_1210,content_val_1212,content_val_1216,content_val_1217,content_val_1218,content_val_1219,content_val_1223,content_val_1225,content_val_1226,content_val_1227,content_val_1229,content_val_123,content_val_1230,content_val_1231,content_val_1232,content_val_1234,content_val_1237,content_val_1238,content_val_124,content_val_1241,content_val_1242,content_val_1243,content_val_1245,content_val_1247,content_val_1248,content_val_1249,content_val_1250,content_val_1252,content_val_1253,content_val_1255,content_val_1257,content_val_1258,content_val_1259,content_val_126,content_val_1261,content_val_1262,content_val_1263,content_val_1264,content_val_1267,content_val_1268,content_val_1269,content_val_127,content_val_1273,content_val_1274,content_val_

In [26]:
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

In [27]:
content_array = content_dummies.drop(columns=['content_id']).values

In [28]:
content_cosine_sim = cosine_similarity(content_array,content_array)

In [29]:
# content_cosine_sim_df =pd.DataFrame(content_cosine_sim)

In [30]:
# content_sim={}
# def get_similarities(row):
#     if row.name%1000==0:
#         print(row.name,end=",")
#     sim_scores = list(enumerate(row.values))
#     sim_scores = sorted(sim_scores,key =lambda x:x[1],reverse= True )
#     sim_scores = sim_scores[:20]
# #     indices = [j[0] for j in sim_scores]
#     content_sim[row.name] = sim_scores
    

In [31]:
# _temp=cosine_dim_df.apply(get_similarities,axis=1)

In [32]:
# import pickle

In [33]:
# #### storing the similarities of the data
# content_sim_file = open('content_sim_file_4.txt', 'wb')
# pickle.dump(content_sim, content_sim_file)
# content_sim_file.close()

In [34]:
# content_dummies['new_content_id']=[ i for i in range(len(content_dummies))]
content_dict={ key:val for key,val in enumerate(content_dummies['content_id'])}
inv_content_dict={ val:key for key,val in enumerate(content_dummies['content_id'])}

In [35]:
relation_grpby.columns

Index(['user_id', 'content_id', 'user_watch_dur_mins', 'date'], dtype='object')

In [36]:
merge_master=relation_grpby.merge(
    user_dummies,on=['user_id']).merge(content_dummies,on=['content_id'])

In [37]:
test_data=pd.read_csv("test.csv")
test_data=test_data.merge(user_dummies)
not_present=test_data[~test_data['user_id'].isin(merge_master['user_id'].unique())]
present = test_data[test_data['user_id'].isin(merge_master['user_id'].unique())]

In [38]:
merge_master['watch_perc'] = merge_master['user_watch_dur_mins']/merge_master['content_duraiton_mins']

In [39]:
merge_master['date_rank'] = merge_master.sort_values(['date'],ascending=False).groupby(['user_id']).cumcount()+1

In [40]:
merge_master['watch_perc_rank'] = merge_master.sort_values(['watch_perc'],ascending=False).groupby(['user_id']).cumcount()+1

In [41]:
merge_master['sum_date_watch_rank'] = merge_master['date_rank']+merge_master['watch_perc_rank']

In [43]:
merge_master.sort_values(['sum_date_watch_rank','rating'],ascending=[True,False],inplace=True)

In [44]:
merge_master_list_grp = merge_master.groupby(['user_id']).agg({'content_id':list}).reset_index()

In [45]:
merge_master_list_grp['content_list']=merge_master_list_grp['content_id'].apply(lambda x: x[:10])

In [47]:
merge_master.sort_values(['date'],ascending=False,inplace=True)

In [48]:
user_last_watched_content = {}
for i,row in merge_master.groupby(['user_id'])['content_id'].first().reset_index().iterrows():
    user_last_watched_content[row['user_id']]=row['content_id']

In [49]:
user_content_list_dict={}
for i,row in merge_master_list_grp.iterrows():
    user_content_list_dict[row['user_id']]=row['content_list']

In [50]:
user_already_viwed_cont_dict = {}
for i,row in merge_master_list_grp.iterrows():
    user_already_viwed_cont_dict[row['user_id']]=row['content_id']

In [51]:
# relation_data.groupby(['user_id'])['content_id'].nunique().reset_index().sort_values(['content_id'],ascending=False).hist()

In [52]:
# merge_master[merge_master['user_id']=='user_19736@domain.com'].sort_values(['date'],ascending=False).head(50)[['content_id','date','watch_perc','date_rank','watch_perc_rank','sum_date_watch_rank']]

In [53]:
user_to_content={}
tmp=0
for user in present['user_id']:
    list_of_top_content = user_content_list_dict[user]
    top_sim_content_dict={}
    visited_vals=[]
    for cont in list_of_top_content:
        cont_idx = inv_content_dict[cont]        
        sim_scores = list(enumerate(content_cosine_sim[cont_idx]))
        sim_scores = sorted(sim_scores,key =lambda x:x[1],reverse= True )
        sim_scores = sim_scores[1:6]
        for i,j in sim_scores:
            cont_id_i = content_dict[i]
#             print(cont_id_i.split("_")[1])
            if cont_id_i not in user_already_viwed_cont_dict[user] and cont_id_i.split("_")[1] not in visited_vals:
                visited_vals.append(cont_id_i.split("_")[1])
                top_sim_content_dict[cont_id_i]=j
                    
    
    output_contents = [k for k,v in sorted(top_sim_content_dict.items(),key=lambda x:x[1],reverse=True)][:10]
    user_to_content[user]=output_contents
    tmp+=1
    
        
    ## add content similar to last wathced content
#     lastwatched_cont_ID = user_last_watched_content[user]
     
#     sim_s = list(enumerate(content_cosine_sim[inv_content_dict[lastwatched_cont_ID]]))
#     sim_s = sorted(sim_s,key =lambda x:x[1],reverse= True )
#     sim_s = sim_s[1:6]
#     for i,j in sim_s:
#         if content_dict[i] not in user_already_viwed_cont_dict[user] and content_dict[i] not in top_sim_content_dict:
#             user_to_content[user].insert(0,content_dict[i])
            
    if tmp%100==0:
        print(tmp,end=",")
    
        
    

100,200,300,400,500,600,700,800,900,1000,

In [54]:
for k,v in user_to_content.items():
    if len(v)<10:
        print(k,v)

user_15509@domain.com ['cont_3179_1_4', 'cont_2201_1_3', 'cont_2362_1_3', 'cont_3421_1_3', 'cont_1417_1_3']
user_17469@domain.com ['cont_3160_3_27', 'cont_2878_13_10', 'cont_2637_3_2', 'cont_56_5_6', 'cont_1063_5_6', 'cont_2161_3_3', 'cont_789_3_4']
user_1375@domain.com ['cont_1223_7_11', 'cont_3874_10_9']
user_10581@domain.com ['cont_1283_9_25', 'cont_3451_2_33', 'cont_4599_1_38', 'cont_3371_1_33']
user_11165@domain.com ['cont_4982_1_1', 'cont_2146_1_1', 'cont_2337_1_1', 'cont_4465_1_1', 'cont_4925_1_1']
user_12662@domain.com ['cont_286_8_27', 'cont_1349_1_26', 'cont_960_5_12', 'cont_3065_12_2', 'cont_3565_8_35', 'cont_4003_31_8', 'cont_2237_14_11', 'cont_3859_1_35']
user_18906@domain.com ['cont_2019_8_10', 'cont_3939_8_15', 'cont_2911_8_10']
user_10778@domain.com ['cont_2489_18_14', 'cont_3183_6_9', 'cont_2911_5_12', 'cont_1584_5_11', 'cont_2552_5_10', 'cont_1341_5_9']
user_12475@domain.com ['cont_321_2_8', 'cont_87_2_7', 'cont_728_2_8']
user_11228@domain.com ['cont_1327_1_14']
user_

In [55]:
# user_to_content

In [56]:
user_dummies.drop(columns=['user_id']).values.T.shape

(37, 13843)

In [57]:
not_present.drop(columns=['user_id']).values.shape

(438, 37)

In [58]:
cosine_sim_user_nt = cosine_similarity(not_present.drop(columns=['user_id']).values,user_dummies.drop(columns=['user_id']).values)

In [59]:
user_sim={}
for i in range(cosine_sim_user_nt.shape[0]):
    sim_scores = list(enumerate(cosine_sim_user_nt[i]))
    sim_scores = sorted(sim_scores,key =lambda x:x[1],reverse= True )
    sim_scores = sim_scores[1:10]
    indices = [(j[0],j[1]) for j in sim_scores]
    user_sim[i]=indices

In [60]:
user_not_present_dict = { key:val for key,val in enumerate(not_present['user_id'])}
user_base_dict = { key:val for key,val in enumerate(user_dummies['user_id'])}

In [61]:
# merge_master.columns

In [62]:
user_to_user = {}
for u in user_sim:
    user_idnotpresent = user_not_present_dict[u]
    user_li = []
    for user_val,score in user_sim[u]:
        user_li.append(user_base_dict[user_val])
    similar_users_content_df = merge_master[merge_master['user_id'].isin(user_li)]
#     print(similar_users_content_df)
    recommended_content= similar_users_content_df.sort_values(['sum_date_watch_rank','rating'],ascending=[True,False])['content_id'].values[:10]
    user_to_user[user_idnotpresent] = list(recommended_content)
    
    

In [63]:
user_to_user

{'user_88625@domain.com': ['cont_4662_2_13',
  'cont_4992_3_16',
  'cont_3767_3_10',
  'cont_2217_2_5',
  'cont_2298_4_22',
  'cont_3515_35_1',
  'cont_4873_6_24',
  'cont_4606_14_4',
  'cont_3955_2_12',
  'cont_1216_8_10'],
 'user_83738@domain.com': ['cont_2282_3_32',
  'cont_1602_5_6',
  'cont_2459_1_13',
  'cont_2401_1_16',
  'cont_1991_4_43',
  'cont_4136_1_24',
  'cont_693_13_2',
  'cont_4688_1_12',
  'cont_3442_16_3',
  'cont_2417_11_19'],
 'user_24220@domain.com': ['cont_1896_5_5',
  'cont_1625_11_20',
  'cont_4702_8_8',
  'cont_973_12_16',
  'cont_1798_9_37',
  'cont_3407_1_3',
  'cont_3256_1_12',
  'cont_4103_10_7',
  'cont_2860_8_1',
  'cont_3371_8_7'],
 'user_53034@domain.com': ['cont_4341_21_4',
  'cont_4021_1_7',
  'cont_882_17_9',
  'cont_2060_3_36',
  'cont_2619_1_1',
  'cont_2707_11_34',
  'cont_1562_1_22',
  'cont_4994_19_30',
  'cont_2194_9_3',
  'cont_1337_6_4'],
 'user_91075@domain.com': ['cont_3352_8_20',
  'cont_4944_1_31',
  'cont_739_1_1',
  'cont_149_8_30',
  '

In [64]:
import copy

In [65]:
final_output = copy.deepcopy(user_to_content)

In [66]:
for key,val in user_to_user.items():
    final_output[key]=list(user_to_user[key])

In [67]:
len(final_output)

1440

In [68]:
# user_to_content

In [69]:
for i,j in final_output.items():
    if len(j)>10:
        print(i,j)

In [78]:
for i,j in final_output.items():
    if len(j)<10:
        print(i,j)
#         final_output[i].extend(final_output[i])

In [80]:
for i,j in final_output.items():
    if len(j)>10:
        print(i,j)
#         final_output[i] = final_output[i][:10]

In [81]:
import json

with open("submit_output_17.json","w") as fp:
    json.dump(final_output,fp,indent=4)

In [196]:
relation_data[relation_data['user_id']=='user_14727@domain.com'].sort_values(['date','content_id'],ascending=False)

,user_id,content_id,duration,date,start_time,end_time,user_watch_dur_mins
614638,user_14727@domain.com,cont_2767_7_6,1440000,2021-05-10,11:15:12,11:39:12,24.0
237055,user_14727@domain.com,cont_135_7_11,1080000,2021-04-26,16:56:19,17:14:19,18.0
1340527,user_14727@domain.com,cont_3939_20_8,2460000,2021-04-22,07:57:58,08:38:58,41.0
1111833,user_14727@domain.com,cont_3313_2_29,1860000,2021-02-21,13:43:49,14:14:49,31.0
14218,user_14727@domain.com,cont_1856_6_7,3900000,2021-02-15,22:10:28,23:15:28,65.0
420972,user_14727@domain.com,cont_3313_3_34,240000,2021-01-20,16:58:32,17:02:32,4.0
5820,user_14727@domain.com,cont_3939_7_11,1680000,2021-01-01,07:21:52,07:49:52,28.0
1241235,user_14727@domain.com,cont_3313_1_37,3180000,2020-12-18,03:24:35,04:17:35,53.0
1567159,user_14727@domain.com,cont_3939_24_15,3720000,2020-11-09,16:55:06,17:57:06,62.0
149260,user_14727@domain.com,cont_2019_1_6,2760000,2020-11-06,04:16:48,05:02:48,46.0


In [191]:
relation_merge_file[relation_merge_file['user_id']=='user_14727@domain.com']

,user_id,content_id,user_watch_dur_mins,user_watch_dur_mins_scl,user_age,gender,location,joining_date,user_yrs,user_age_bin,user_join_yrs_scl,content_type,language,genre,duration,release_date,rating,episode_count,season_count,content_duraiton_mins,content_yrs,content_duraiton_mins_scl,content_yrs_scl,rating_scl,episode_count_scl,season_count_scl,watch_percent,watch_perc_scl
86959,user_14727@domain.com,cont_3939_7_11,28.0,0.103053,30,F,West Bengal,2020-07-21,1.67,1,0.135385,series,bengali,action,4200000,1994-06-03,9,11,7,70.0,27.80,0.375000,0.879219,0.9,0.183333,0.159091,0.400000,0.131752
191384,user_14727@domain.com,cont_3939_24_15,62.0,0.232824,30,F,West Bengal,2020-07-21,1.67,1,0.135385,series,bengali,action,4200000,2002-12-06,9,15,24,70.0,19.30,0.375000,0.597948,0.9,0.250000,0.545455,0.885714,0.293952
247201,user_14727@domain.com,cont_2019_1_6,46.0,0.171756,30,F,West Bengal,2020-07-21,1.67,1,0.135385,series,bengali,drama,3360000,2017-01-15,10,6,1,56.0,5.18,0.298913,0.130708,1.0,0.100000,0.022727,0.821429,0.272484
249161,user_14727@domain.com,cont_3313_1_37,53.0,0.198473,30,F,West Bengal,2020-07-21,1.67,1,0.135385,series,bengali,comedy,3180000,2017-02-22,7,37,1,53.0,5.08,0.282609,0.127399,0.7,0.616667,0.022727,1.000000,0.332117
250149,user_14727@domain.com,cont_3313_3_34,4.0,0.011450,30,F,West Bengal,2020-07-21,1.67,1,0.135385,series,bengali,comedy,3060000,2020-05-25,7,34,3,51.0,1.83,0.271739,0.019854,0.7,0.566667,0.068182,0.078431,0.024367
251824,user_14727@domain.com,cont_3939_18_13,27.0,0.099237,30,F,West Bengal,2020-07-21,1.67,1,0.135385,series,bengali,action,4320000,1999-11-28,9,13,18,72.0,22.32,0.385870,0.697882,0.9,0.216667,0.409091,0.375000,0.123403
259016,user_14727@domain.com,cont_135_5_15,35.0,0.129771,30,F,West Bengal,2020-07-21,1.67,1,0.135385,series,bengali,comedy,3120000,2000-10-28,6,15,5,52.0,21.40,0.277174,0.667439,0.6,0.250000,0.113636,0.673077,0.222944
259303,user_14727@domain.com,cont_135_7_11,18.0,0.064885,30,F,West Bengal,2020-07-21,1.67,1,0.135385,series,bengali,comedy,3180000,2001-09-05,6,11,7,53.0,20.55,0.282609,0.639312,0.6,0.183333,0.159091,0.339623,0.111589
260721,user_14727@domain.com,cont_1856_1_8,63.0,0.236641,30,F,West Bengal,2020-07-21,1.67,1,0.135385,series,bengali,drama,4080000,2000-12-29,9,8,1,68.0,21.23,0.364130,0.661813,0.9,0.133333,0.022727,0.926471,0.307562
261529,user_14727@domain.com,cont_1856_6_7,65.0,0.244275,30,F,West Bengal,2020-07-21,1.67,1,0.135385,series,bengali,drama,3960000,2008-05-09,9,7,6,66.0,13.87,0.353261,0.418266,0.9,0.116667,0.136364,0.984848,0.327057


##############################################################################################################################
## Below code is an experimental approach
#### Below code is the using collaborative filtering using a Embeddings and target variable is implicit factor of user-content watch % and ratings of the content
#### implicitly getting the target variable as rating from watch % of user to the content

In [27]:
merge_master['watch_perc']=(merge_master['user_watch_dur_mins']/merge_master['content_duraiton_mins'])*100

In [28]:
# check
merge_master[(merge_master['user_id']=='user_99974@domain.com' ) & (merge_master['content_id']=='cont_2311_5_19')]

,user_id,content_id,user_watch_dur_mins,user_age,gender,location,joining_date,user_yrs,gender_id,locations_id,user_id_map,user_age_bin,user_join_yrs_scl,content_type,language,genre,duration,release_date,rating,episode_count,season_count,total_episodes,content_duraiton_mins,content_yrs,content_type_id,language_id,genre_id,total_episodes_scl,content_duraiton_mins_scl,content_yrs_scl,rating_scl,content_id_map,watch_perc
188339,user_99974@domain.com,cont_2311_5_19,108.0,20,M,Assam,2019-04-29,2.9,0,11,5648,0,0.516923,series,bengali,comedy,3600000,2003-03-20,4,19,5,95,60.0,19.01,0,10,1,0.110723,0.320652,0.588488,0.4,32833,180.0


In [34]:
merge_master['watch_bins']=pd.cut(merge_master['watch_perc'], [0,50,300], labels=[0,1])

In [35]:
merge_master['watch_perc_scl'] = minmaxsclar.fit_transform(merge_master['watch_perc'].values.reshape(-1,1))

In [36]:
merge_master['avg_ratin_viewtime'] = merge_master.apply( lambda x: (x['watch_perc_scl']+x['rating_scl'])/2,axis=1)

In [37]:
merge_master['user_join_yrs_scl']=minmaxsclar.fit_transform(merge_master['user_yrs'].values.reshape(-1,1))

In [38]:
merge_master.columns

Index(['user_id', 'content_id', 'user_watch_dur_mins', 'user_age', 'gender',
       'location', 'joining_date', 'user_yrs', 'gender_id', 'locations_id',
       'user_id_map', 'user_age_bin', 'user_join_yrs_scl', 'content_type',
       'language', 'genre', 'duration', 'release_date', 'rating',
       'episode_count', 'season_count', 'total_episodes',
       'content_duraiton_mins', 'content_yrs', 'content_type_id',
       'language_id', 'genre_id', 'total_episodes_scl',
       'content_duraiton_mins_scl', 'content_yrs_scl', 'rating_scl',
       'content_id_map', 'watch_perc', 'watch_bins', 'watch_perc_scl',
       'avg_ratin_viewtime'],
      dtype='object')

In [40]:
req_subset = merge_master[['user_id_map','content_id_map','content_type_id', 'gender_id', 'locations_id', 'user_age_bin',
              'language_id', 'genre_id', 'total_episodes_scl','content_duraiton_mins_scl', 'user_join_yrs_scl',
              'content_yrs_scl', 'rating_scl','watch_bins']]

In [41]:
# min_watch_perc = min(data_df['%watch'])
# max_watch_perc = max(data_df['%watch'])

In [42]:
from keras.models import Model
from keras.layers import Input
from keras.layers.core import Reshape, Dropout, Dense
from keras.layers.merge import Multiply, Dot
from keras.layers.embeddings import Embedding
from keras.layers.merge import Concatenate
from keras import optimizers

In [43]:
import numpy as np

In [44]:
# data_df['user_id_map'],data_df['content_id_map'],data_df['content_type_id'],data_df['language_id'],data_df['genre_id'],
# data_df['gender_id'],data_df['locations_id'],data_df['content_duration_mins_scaled'],
# data_df['total_episodes_scaled'],data_df['%watch']

In [45]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.regularizers import l2

In [46]:
# X_train, X_test, y_train, y_test = train_test_split(pd.concat([data_df[cat_cols],data_df[numeric_cols]],axis=1), 
#                                                     data_df['%watch'], test_size=0.1, random_state=42)

In [47]:
req_subset.columns

Index(['user_id_map', 'content_id_map', 'content_type_id', 'gender_id',
       'locations_id', 'user_age_bin', 'language_id', 'genre_id',
       'total_episodes_scl', 'content_duraiton_mins_scl', 'user_join_yrs_scl',
       'content_yrs_scl', 'rating_scl', 'watch_bins'],
      dtype='object')

In [48]:
user_size = max(req_subset['user_id_map'])+1
content_size = max(req_subset['content_id_map'])+1
content_type_size = max(req_subset['content_type_id'])+1
gender_size = max(req_subset['gender_id'])+1
location_size = max(req_subset['locations_id'])+1
user_age_size = max(req_subset['user_age_bin'])+1
lang_size = max(req_subset['language_id'])+1
genre_size = max(req_subset['genre_id'])+1

In [71]:

user_input = Input(shape=[1], name='user')
content_input = Input(shape=[1], name='content')
content_type_input = Input(shape=[1], name='content_type')
gender_input = Input(shape=[1], name='gender')
location_input = Input(shape=[1], name='location')
user_age_input = Input(shape=[1], name='user_age')
lang_input = Input(shape=[1], name='lang')
genre_input = Input(shape=[1], name='genre')


# numeric data
# content_duration_mins_scaled , total_episodes_scaled , %watch
total_episodes_input = Input(shape=[1],name='total_episodes')
content_duration_input = Input(shape=[1],name='content_duration')
user_join_yrs_input = Input(shape=[1],name='user_join_yrs')
content_yrs_input = Input(shape=[1],name='content_yrs')
content_rating_input = Input(shape=[1],name='content_rating')


# embedding dim for categorical inputs
embedding_user_size = 60
embedding_content_size = 60
embedding_content_type_size = 5
embedding_gender = 2
embedding_location = 15
embedding_user_age = 5
embedding_language = 11
embedding_genre = 15


user_embedding = Embedding(output_dim=embedding_user_size, input_dim=user_size,
                           input_length=1, name='user_embedding',embeddings_regularizer = l2(1e-4))(user_input)

content_embedding = Embedding(output_dim=embedding_content_size, input_dim=content_size,
                              input_length=1, name='content_embedding',embeddings_regularizer = l2(1e-4))(content_input)

content_type_embedding = Embedding(output_dim=embedding_content_type_size, input_dim=content_type_size,
                                   input_length=1, name='content_type_embedding',embeddings_regularizer = l2(1e-4))(content_type_input)

gender_embedding = Embedding(output_dim=embedding_gender, input_dim=gender_size,
                                   input_length=1, name='gender_embedding',embeddings_regularizer = l2(1e-4))(gender_input)

location_embedding = Embedding(output_dim=embedding_location, input_dim=location_size,
                                   input_length=1, name='location_embedding',embeddings_regularizer = l2(1e-4))(location_input)

user_age_embedding = Embedding(output_dim=embedding_user_age, input_dim=user_age_size,
                                   input_length=1, name='user_age_embedding',embeddings_regularizer = l2(1e-4))(user_age_input)

lang_embedding = Embedding(output_dim=embedding_language, input_dim=lang_size,
                                   input_length=1, name='lang_embedding',embeddings_regularizer = l2(1e-4))(lang_input)

genre_embedding = Embedding(output_dim=embedding_genre, input_dim=genre_size,
                                   input_length=1, name='genre_embedding',embeddings_regularizer = l2(1e-4))(genre_input)




user_vecs = Reshape([embedding_user_size])(user_embedding)
content_vecs = Reshape([embedding_content_size])(content_embedding)
content_type_vecs = Reshape([embedding_content_type_size])(content_type_embedding)
gender_vecs = Reshape([embedding_gender])(gender_embedding)
location_vecs = Reshape([embedding_location])(location_embedding)
user_age_vecs = Reshape([embedding_user_age])(user_age_embedding)
lang_vecs = Reshape([embedding_language])(lang_embedding)
genre_vecs = Reshape([embedding_genre])(genre_embedding)



input_vecs = Concatenate()([user_vecs,content_vecs,content_type_vecs,gender_vecs,location_vecs,user_age_vecs,
                            lang_vecs,genre_vecs,total_episodes_input,content_duration_input,user_join_yrs_input,
                            content_yrs_input,content_rating_input])


input_vecs = Dropout(0.2)(input_vecs) 

x = Dense(256, activation='relu')(input_vecs)
x = Dropout(0.2)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
# y = Dense(1, activation='relu')(x)
# y = Dense(1)(x)
# y = Lambda(lambda x: x * (max_rating - min_rating) + min_rating)(x)
y = Dense(1, activation='sigmoid', name = 'prediction')(x)



model = Model(inputs=[user_input,content_input,content_type_input,gender_input,location_input,user_age_input,lang_input,
                      genre_input, total_episodes_input,content_duration_input,user_join_yrs_input,
                      content_yrs_input,content_rating_input], outputs=y)
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])

In [72]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user (InputLayer)               [(None, 1)]          0                                            
__________________________________________________________________________________________________
content (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
content_type (InputLayer)       [(None, 1)]          0                                            
__________________________________________________________________________________________________
gender (InputLayer)             [(None, 1)]          0                                            
____________________________________________________________________________________________

In [73]:
req_subset.columns

Index(['user_id_map', 'content_id_map', 'content_type_id', 'gender_id',
       'locations_id', 'user_age_bin', 'language_id', 'genre_id',
       'total_episodes_scl', 'content_duraiton_mins_scl', 'user_join_yrs_scl',
       'content_yrs_scl', 'rating_scl', 'watch_bins'],
      dtype='object')

In [74]:
# [req_subset['user_id_map'], req_subset['content_id_map'], req_subset['content_type_id'], req_subset['gender_id'],
#        req_subset['locations_id'], req_subset['user_age_bin'], req_subset['language_id'], req_subset['genre_id'],
#        req_subset['total_episodes_scl'], req_subset['content_duraiton_mins_scl'], req_subset['user_join_yrs_scl'],
#         req_subset['content_yrs_scl'], req_subset['rating_scl'] ],req_subset['avg_ratin_viewtime']

In [75]:
# req_subset[['user_id_map', 'content_id_map', 'content_type_id', 'gender_id',
#        'locations_id', 'user_age_bin', 'language_id', 'genre_id',
#        'total_episodes_scl', 'content_duraiton_mins_scl', 'user_join_yrs_scl',
#        'content_yrs_scl', 'rating_scl']].reset_index(drop=True).values

In [76]:
req_subset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1639750 entries, 0 to 1639749
Data columns (total 14 columns):
 #   Column                     Non-Null Count    Dtype   
---  ------                     --------------    -----   
 0   user_id_map                1639750 non-null  int64   
 1   content_id_map             1639750 non-null  int64   
 2   content_type_id            1639750 non-null  int64   
 3   gender_id                  1639750 non-null  int64   
 4   locations_id               1639750 non-null  int64   
 5   user_age_bin               1639750 non-null  int32   
 6   language_id                1639750 non-null  int64   
 7   genre_id                   1639750 non-null  int64   
 8   total_episodes_scl         1639750 non-null  float64 
 9   content_duraiton_mins_scl  1639750 non-null  float64 
 10  user_join_yrs_scl          1639750 non-null  float64 
 11  content_yrs_scl            1639750 non-null  float64 
 12  rating_scl                 1639750 non-null  float64 
 1

In [ ]:

model.fit([req_subset['user_id_map'], req_subset['content_id_map'], req_subset['content_type_id'], req_subset['gender_id'],
       req_subset['locations_id'], req_subset['user_age_bin'], req_subset['language_id'], req_subset['genre_id'],
       req_subset['total_episodes_scl'], req_subset['content_duraiton_mins_scl'], req_subset['user_join_yrs_scl'],
        req_subset['content_yrs_scl'], req_subset['rating_scl'] ],
          req_subset['watch_bins']
                    , batch_size=64, epochs=2
#                     , validation_split=0.1
                    , shuffle=True)

Epoch 1/2
 1521/25622 [>.............................] - ETA: 22:12 - loss: 0.7029 - accuracy: 0.5035

In [ ]:
X_test = req_subset.sample(50,random_state=30)

In [ ]:
X_test

In [ ]:
outpt=model.predict([req_subset['user_id_map'], req_subset['content_id_map'], req_subset['content_type_id'], req_subset['gender_id'],
       req_subset['locations_id'], req_subset['user_age_bin'], req_subset['language_id'], req_subset['genre_id'],
       req_subset['total_episodes_scl'], req_subset['content_duraiton_mins_scl'], req_subset['user_join_yrs_scl'],
        req_subset['content_yrs_scl'], req_subset['rating_scl'] ])

In [67]:
X_test['outpt_watch_time'] = outpt

In [81]:
test_data=pd.read_csv("test.csv")

In [84]:
test_data=test_data.merge(user_data)
not_present=test_data[~test_data['user_id'].isin(merge_master['user_id'].unique())]
present = test_data[test_data['user_id'].isin(merge_master['user_id'].unique())]

In [87]:
present

,user_id,user_age,gender,location,joining_date,user_yrs,gender_id,locations_id,user_id_map,user_age_bin,user_join_yrs_scl
0,user_18085@domain.com,40,M,Telangana,2018-12-08,3.29,0,8,1225,2,0.636923
1,user_16044@domain.com,43,F,Odisa,2017-10-13,4.44,1,13,5937,3,0.990769
2,user_13110@domain.com,46,F,West Bengal,2018-10-21,3.42,1,2,7473,3,0.676923
3,user_18909@domain.com,32,F,Goa,2019-01-05,3.21,1,0,2609,2,0.612308
4,user_15509@domain.com,31,F,Maharashtra,2020-04-26,1.90,1,7,11388,2,0.209231
...,...,...,...,...,...,...,...,...,...,...,...
1434,user_10398@domain.com,55,M,Odisa,2017-10-07,4.46,0,13,3825,4,0.996923
1435,user_1609@domain.com,38,M,Punjab,2018-03-17,4.02,0,9,7078,2,0.861538
1436,user_13422@domain.com,43,M,Punjab,2018-04-02,3.97,0,9,6964,3,0.846154
1438,user_10393@domain.com,59,M,Karnataka,2020-09-18,1.51,0,5,2227,4,0.089231


In [88]:
content_data

,content_id,content_type,language,genre,duration,release_date,rating,episode_count,season_count,total_episodes,content_duraiton_mins,content_yrs,content_type_id,language_id,genre_id,total_episodes_scl,content_duraiton_mins_scl,content_yrs_scl,rating_scl,content_id_map
0,cont_475_19_32,series,english,drama,4980000,2018-07-01,10,32,19,608,83.0,3.72,0,0,0,0.708625,0.445652,0.082699,1.0,0
1,cont_2185_15_21,series,english,drama,3000000,2016-03-29,4,21,15,315,50.0,5.98,0,0,0,0.367133,0.266304,0.157459,0.4,1
2,cont_4857_13_28,series,tamil,comedy,3120000,2006-03-06,8,28,13,364,52.0,16.05,0,1,1,0.424242,0.277174,0.490572,0.8,2
3,cont_3340_1_5,sports,hindi,cricket,9900000,2009-01-10,0,5,1,5,165.0,13.20,1,2,2,0.005828,0.891304,0.396295,0.0,3
4,cont_1664_10_29,series,hindi,action,3660000,2020-05-25,2,29,10,290,61.0,1.82,0,2,3,0.337995,0.326087,0.019848,0.2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48640,cont_4218_6_15,series,hindi,drama,3360000,2015-02-04,6,15,6,90,56.0,7.13,0,2,0,0.104895,0.298913,0.195501,0.6,48640
48641,cont_2533_1_14,series,marathi,sci-fi,3120000,2002-01-15,4,14,1,14,52.0,20.18,0,3,6,0.016317,0.277174,0.627192,0.4,48641
48642,cont_4606_33_5,series,hindi,drama,3180000,2006-02-18,6,5,33,165,53.0,16.09,0,2,0,0.192308,0.282609,0.491895,0.6,48642
48643,cont_3708_9_1,series,english,drama,4020000,2010-04-12,5,1,9,9,67.0,11.94,0,0,0,0.010490,0.358696,0.354615,0.5,48643


In [98]:
pd.DataFrame(present.loc[0]).T

,user_id,user_age,gender,location,joining_date,user_yrs,gender_id,locations_id,user_id_map,user_age_bin,user_join_yrs_scl
0,user_18085@domain.com,40,M,Telangana,2018-12-08,3.29,0,8,1225,2,0.636923


In [100]:
def cal_dataframes(row):
    user_id_map=row['user_id_map']
    viewed_content = merge_master[merge_master['user_id_map']==user_id_map]['content_id_map'].unique()
    new_df = content_data[~content_data['content_id_map'].isin(viewed_content)]
    new_df['key']=0
    row['key']=0
    new_df = pd.DataFrame(row).T.merge(new_df, on='key', how='outer')
    new_df.drop(columns=['key'],inplace=True)
    return new_df
    
predict_df = []
for index, row in present.iterrows():
    predict_df.append(cal_dataframes(row))
    

<ipython-input-100-a90f172f651e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['key']=0
<ipython-input-100-a90f172f651e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['key']=0
<ipython-input-100-a90f172f651e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

<ipython-input-100-a90f172f651e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['key']=0
<ipython-input-100-a90f172f651e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['key']=0
<ipython-input-100-a90f172f651e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

<ipython-input-100-a90f172f651e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['key']=0
<ipython-input-100-a90f172f651e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['key']=0
<ipython-input-100-a90f172f651e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

<ipython-input-100-a90f172f651e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['key']=0
<ipython-input-100-a90f172f651e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['key']=0
<ipython-input-100-a90f172f651e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

<ipython-input-100-a90f172f651e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['key']=0
<ipython-input-100-a90f172f651e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['key']=0
<ipython-input-100-a90f172f651e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

...

<ipython-input-100-a90f172f651e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['key']=0
<ipython-input-100-a90f172f651e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['key']=0
<ipython-input-100-a90f172f651e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

<ipython-input-100-a90f172f651e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['key']=0
<ipython-input-100-a90f172f651e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['key']=0
<ipython-input-100-a90f172f651e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

<ipython-input-100-a90f172f651e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['key']=0
<ipython-input-100-a90f172f651e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['key']=0
<ipython-input-100-a90f172f651e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

<ipython-input-100-a90f172f651e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['key']=0
<ipython-input-100-a90f172f651e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['key']=0
<ipython-input-100-a90f172f651e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

KeyboardInterrupt: 

In [103]:
present_to_predict=pd.concat(predict_df)

In [105]:
present_to_predict['user_id'].unique()

array(['user_18085@domain.com', 'user_16044@domain.com',
       'user_13110@domain.com', 'user_18909@domain.com',
       'user_15509@domain.com', 'user_10536@domain.com',
       'user_14011@domain.com', 'user_11121@domain.com',
       'user_14901@domain.com', 'user_18370@domain.com',
       'user_1228@domain.com', 'user_151@domain.com',
       'user_17314@domain.com', 'user_17146@domain.com',
       'user_1819@domain.com', 'user_1533@domain.com',
       'user_14564@domain.com', 'user_11950@domain.com',
       'user_14154@domain.com', 'user_11602@domain.com',
       'user_10792@domain.com', 'user_15529@domain.com',
       'user_10076@domain.com', 'user_14100@domain.com',
       'user_16037@domain.com', 'user_18740@domain.com',
       'user_1123@domain.com', 'user_13172@domain.com',
       'user_12511@domain.com', 'user_10735@domain.com',
       'user_14727@domain.com', 'user_12683@domain.com',
       'user_10853@domain.com', 'user_16544@domain.com',
       'user_13117@domain.com', 'user

In [108]:
def run_predict(temp_df):
    outpt=model.predict([temp_df['user_id_map'], temp_df['content_id_map'], temp_df['content_type_id'], temp_df['gender_id'],
       temp_df['locations_id'], temp_df['user_age_bin'], temp_df['language_id'], temp_df['genre_id'],
       temp_df['total_episodes_scl'], temp_df['content_duraiton_mins_scl'], temp_df['user_join_yrs_scl'],
        temp_df['content_yrs_scl'], temp_df['rating_scl'] ])
    temp_df['output_rating'] = outpt
    top_10 = temp_df.sort_values(['output_rating'],ascending=False)[:10]['content_id'].values
    return top_10

In [109]:
present_to_predict.columns

Index(['user_id', 'user_age', 'gender', 'location', 'joining_date', 'user_yrs',
       'gender_id', 'locations_id', 'user_id_map', 'user_age_bin',
       'user_join_yrs_scl', 'content_id', 'content_type', 'language', 'genre',
       'duration', 'release_date', 'rating', 'episode_count', 'season_count',
       'total_episodes', 'content_duraiton_mins', 'content_yrs',
       'content_type_id', 'language_id', 'genre_id', 'total_episodes_scl',
       'content_duraiton_mins_scl', 'content_yrs_scl', 'rating_scl',
       'content_id_map'],
      dtype='object')

In [113]:
present_to_predict['user_id_map'] = present_to_predict['user_id_map'].astype(int)
present_to_predict['content_id_map'] = present_to_predict['content_id_map'].astype(int)
present_to_predict['content_type_id'] = present_to_predict['content_type_id'].astype(int)
present_to_predict['gender_id'] = present_to_predict['gender_id'].astype(int)
present_to_predict['locations_id'] = present_to_predict['locations_id'].astype(int)
present_to_predict['user_age_bin'] = present_to_predict['user_age_bin'].astype(int)
present_to_predict['language_id'] = present_to_predict['language_id'].astype(int)
present_to_predict['genre_id'] = present_to_predict['genre_id'].astype(int)
present_to_predict['total_episodes_scl'] = present_to_predict['total_episodes_scl'].astype(float)
present_to_predict['content_duraiton_mins_scl'] = present_to_predict['content_duraiton_mins_scl'].astype(float)
present_to_predict['user_join_yrs_scl'] = present_to_predict['user_join_yrs_scl'].astype(float)
present_to_predict['content_yrs_scl'] = present_to_predict['content_yrs_scl'].astype(float)
present_to_predict['rating_scl'] = present_to_predict['rating_scl'].astype(float)


In [114]:
# present_to_predict[['user_id_map','content_id_map','content_type_id','gender_id','locations_id','user_age_bin',
#                     'language_id','genre_id',
#  'total_episodes_scl','content_duraiton_mins_scl','user_join_yrs_scl','content_yrs_scl','rating_scl']]

In [115]:
output_dict = {}
for user_id_to_pred in present_to_predict['user_id'].unique():
    temp_df = present_to_predict[present_to_predict['user_id']==user_id_to_pred]
    output_dict[user_id_to_pred] = run_predict(temp_df)

<ipython-input-108-4ee19897d8a3>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['output_rating'] = outpt


In [116]:
output_dict

{'user_18085@domain.com': array(['cont_4308_1_12', 'cont_4308_2_12', 'cont_663_1_6',
        'cont_2019_1_2', 'cont_663_1_8', 'cont_807_1_2', 'cont_4308_3_17',
        'cont_4308_4_8', 'cont_3552_1_2', 'cont_4308_1_14'], dtype=object),
 'user_16044@domain.com': array(['cont_4308_1_12', 'cont_663_1_8', 'cont_4308_2_12', 'cont_663_1_6',
        'cont_2019_1_2', 'cont_4308_4_8', 'cont_4308_1_7', 'cont_807_1_2',
        'cont_3552_1_2', 'cont_4308_2_16'], dtype=object),
 'user_13110@domain.com': array(['cont_4308_1_12', 'cont_4308_2_12', 'cont_2019_1_2',
        'cont_4308_3_17', 'cont_663_1_6', 'cont_4308_4_8', 'cont_4308_1_7',
        'cont_807_1_2', 'cont_663_1_8', 'cont_4308_1_14'], dtype=object),
 'user_18909@domain.com': array(['cont_4308_1_12', 'cont_2019_1_2', 'cont_4308_4_8', 'cont_807_1_2',
        'cont_4308_3_17', 'cont_4308_2_12', 'cont_663_1_6',
        'cont_3552_1_2', 'cont_663_1_8', 'cont_4308_1_7'], dtype=object),
 'user_15509@domain.com': array(['cont_4308_2_12', 'cont_6

In [118]:
present

,user_id,user_age,gender,location,joining_date,user_yrs,gender_id,locations_id,user_id_map,user_age_bin,user_join_yrs_scl
0,user_18085@domain.com,40,M,Telangana,2018-12-08,3.29,0,8,1225,2,0.636923
1,user_16044@domain.com,43,F,Odisa,2017-10-13,4.44,1,13,5937,3,0.990769
2,user_13110@domain.com,46,F,West Bengal,2018-10-21,3.42,1,2,7473,3,0.676923
3,user_18909@domain.com,32,F,Goa,2019-01-05,3.21,1,0,2609,2,0.612308
4,user_15509@domain.com,31,F,Maharashtra,2020-04-26,1.90,1,7,11388,2,0.209231
...,...,...,...,...,...,...,...,...,...,...,...
1434,user_10398@domain.com,55,M,Odisa,2017-10-07,4.46,0,13,3825,4,0.996923
1435,user_1609@domain.com,38,M,Punjab,2018-03-17,4.02,0,9,7078,2,0.861538
1436,user_13422@domain.com,43,M,Punjab,2018-04-02,3.97,0,9,6964,3,0.846154
1438,user_10393@domain.com,59,M,Karnataka,2020-09-18,1.51,0,5,2227,4,0.089231


In [ ]:
## using Regression as model approach to predict the watch%

In [ ]:
merge_master=relation_grpby.merge(user_data,on=['user_id']).merge(content_data,on=['content_id'])

In [ ]:
merge_master['watch_perc']=(merge_master['user_watch_dur_mins']/merge_master['content_duraiton_mins'])*100

In [ ]:
# check
merge_master[(merge_master['user_id']=='user_99974@domain.com' ) & (merge_master['content_id']=='cont_2311_5_19')]

In [ ]:
merge_master['watch_perc_scl'] = minmaxsclar.fit_transform(merge_master['watch_perc'].values.reshape(-1,1))

In [ ]:
merge_master['avg_ratin_viewtime'] = merge_master.apply( lambda x: (x['watch_perc_scl']+x['rating_scl'])/2,axis=1)

In [ ]:
merge_master['user_join_yrs_scl']=minmaxsclar.fit_transform(merge_master['user_yrs'].values.reshape(-1,1))

In [ ]:
req_subset = merge_master[['user_id_map','content_id_map','content_type_id', 'gender_id', 'locations_id', 'user_age_bin',
              'language_id', 'genre_id', 'total_episodes_scl','content_duraiton_mins_scl', 'user_join_yrs_scl',
              'content_yrs_scl', 'rating_scl','avg_ratin_viewtime']]

In [ ]:
from keras.models import Model
from keras.layers import Input
from keras.layers.core import Reshape, Dropout, Dense
from keras.layers.merge import Multiply, Dot
from keras.layers.embeddings import Embedding
from keras.layers.merge import Concatenate
from keras import optimizers
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.regularizers import l2

In [ ]:

user_input = Input(shape=[1], name='user')
content_input = Input(shape=[1], name='content')
content_type_input = Input(shape=[1], name='content_type')
gender_input = Input(shape=[1], name='gender')
location_input = Input(shape=[1], name='location')
user_age_input = Input(shape=[1], name='user_age')
lang_input = Input(shape=[1], name='lang')
genre_input = Input(shape=[1], name='genre')


# numeric data
# content_duration_mins_scaled , total_episodes_scaled , %watch
total_episodes_input = Input(shape=[1],name='total_episodes')
content_duration_input = Input(shape=[1],name='content_duration')
user_join_yrs_input = Input(shape=[1],name='user_join_yrs')
content_yrs_input = Input(shape=[1],name='content_yrs')
content_rating_input = Input(shape=[1],name='content_rating')


# embedding dim for categorical inputs
embedding_user_size = 60
embedding_content_size = 60
embedding_content_type_size = 5
embedding_gender = 2
embedding_location = 15
embedding_user_age = 5
embedding_language = 11
embedding_genre = 15


user_embedding = Embedding(output_dim=embedding_user_size, input_dim=user_size,
                           input_length=1, name='user_embedding',embeddings_regularizer = l2(1e-4))(user_input)

content_embedding = Embedding(output_dim=embedding_content_size, input_dim=content_size,
                              input_length=1, name='content_embedding',embeddings_regularizer = l2(1e-4))(content_input)

content_type_embedding = Embedding(output_dim=embedding_content_type_size, input_dim=content_type_size,
                                   input_length=1, name='content_type_embedding',embeddings_regularizer = l2(1e-4))(content_type_input)

gender_embedding = Embedding(output_dim=embedding_gender, input_dim=gender_size,
                                   input_length=1, name='gender_embedding',embeddings_regularizer = l2(1e-4))(gender_input)

location_embedding = Embedding(output_dim=embedding_location, input_dim=location_size,
                                   input_length=1, name='location_embedding',embeddings_regularizer = l2(1e-4))(location_input)

user_age_embedding = Embedding(output_dim=embedding_user_age, input_dim=user_age_size,
                                   input_length=1, name='user_age_embedding',embeddings_regularizer = l2(1e-4))(user_age_input)

lang_embedding = Embedding(output_dim=embedding_language, input_dim=lang_size,
                                   input_length=1, name='lang_embedding',embeddings_regularizer = l2(1e-4))(lang_input)

genre_embedding = Embedding(output_dim=embedding_genre, input_dim=genre_size,
                                   input_length=1, name='genre_embedding',embeddings_regularizer = l2(1e-4))(genre_input)




user_vecs = Reshape([embedding_user_size])(user_embedding)
content_vecs = Reshape([embedding_content_size])(content_embedding)
content_type_vecs = Reshape([embedding_content_type_size])(content_type_embedding)
gender_vecs = Reshape([embedding_gender])(gender_embedding)
location_vecs = Reshape([embedding_location])(location_embedding)
user_age_vecs = Reshape([embedding_user_age])(user_age_embedding)
lang_vecs = Reshape([embedding_language])(lang_embedding)
genre_vecs = Reshape([embedding_genre])(genre_embedding)



input_vecs = Concatenate()([user_vecs,content_vecs,content_type_vecs,gender_vecs,location_vecs,user_age_vecs,
                            lang_vecs,genre_vecs,total_episodes_input,content_duration_input,user_join_yrs_input,
                            content_yrs_input,content_rating_input])


input_vecs = Dropout(0.2)(input_vecs) 

x = Dense(256, activation='relu')(input_vecs)
x = Dropout(0.2)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
y = Dense(1, activation='relu')(x)



model = Model(inputs=[user_input,content_input,content_type_input,gender_input,location_input,user_age_input,lang_input,
                      genre_input, total_episodes_input,content_duration_input,user_join_yrs_input,
                      content_yrs_input,content_rating_input], outputs=y)
model.compile(optimizer='adam', loss='mse')

In [ ]:

model.fit([req_subset['user_id_map'], req_subset['content_id_map'], req_subset['content_type_id'], req_subset['gender_id'],
       req_subset['locations_id'], req_subset['user_age_bin'], req_subset['language_id'], req_subset['genre_id'],
       req_subset['total_episodes_scl'], req_subset['content_duraiton_mins_scl'], req_subset['user_join_yrs_scl'],
        req_subset['content_yrs_scl'], req_subset['rating_scl'] ],
          req_subset['avg_ratin_viewtime']
                    , batch_size=64, epochs=2
#                     , validation_split=0.1
                    , shuffle=True)

In [ ]:
test_data=pd.read_csv("test.csv")
test_data=test_data.merge(user_data)
not_present=test_data[~test_data['user_id'].isin(merge_master['user_id'].unique())]
present = test_data[test_data['user_id'].isin(merge_master['user_id'].unique())]

In [ ]:
def cal_dataframes(row):
    user_id_map=row['user_id_map']
    viewed_content = merge_master[merge_master['user_id_map']==user_id_map]['content_id_map'].unique()
    new_df = content_data[~content_data['content_id_map'].isin(viewed_content)]
    new_df['key']=0
    row['key']=0
    new_df = pd.DataFrame(row).T.merge(new_df, on='key', how='outer')
    new_df.drop(columns=['key'],inplace=True)
    return new_df
    
predict_df = []
for index, row in present.iterrows():
    predict_df.append(cal_dataframes(row))

present_to_predict=pd.concat(predict_df)
del predict_df

In [ ]:
def run_predict(temp_df):
    outpt=model.predict([temp_df['user_id_map'], temp_df['content_id_map'], temp_df['content_type_id'], temp_df['gender_id'],
       temp_df['locations_id'], temp_df['user_age_bin'], temp_df['language_id'], temp_df['genre_id'],
       temp_df['total_episodes_scl'], temp_df['content_duraiton_mins_scl'], temp_df['user_join_yrs_scl'],
        temp_df['content_yrs_scl'], temp_df['rating_scl'] ])
    temp_df['output_rating'] = outpt
    top_10 = temp_df.sort_values(['output_rating'],ascending=False)[:10]['content_id'].values
    return top_10

present_to_predict['user_id_map'] = present_to_predict['user_id_map'].astype(int)
present_to_predict['content_id_map'] = present_to_predict['content_id_map'].astype(int)
present_to_predict['content_type_id'] = present_to_predict['content_type_id'].astype(int)
present_to_predict['gender_id'] = present_to_predict['gender_id'].astype(int)
present_to_predict['locations_id'] = present_to_predict['locations_id'].astype(int)
present_to_predict['user_age_bin'] = present_to_predict['user_age_bin'].astype(int)
present_to_predict['language_id'] = present_to_predict['language_id'].astype(int)
present_to_predict['genre_id'] = present_to_predict['genre_id'].astype(int)
present_to_predict['total_episodes_scl'] = present_to_predict['total_episodes_scl'].astype(float)
present_to_predict['content_duraiton_mins_scl'] = present_to_predict['content_duraiton_mins_scl'].astype(float)
present_to_predict['user_join_yrs_scl'] = present_to_predict['user_join_yrs_scl'].astype(float)
present_to_predict['content_yrs_scl'] = present_to_predict['content_yrs_scl'].astype(float)
present_to_predict['rating_scl'] = present_to_predict['rating_scl'].astype(float)


In [ ]:
output_dict = {}
for user_id_to_pred in present_to_predict['user_id'].unique():
    temp_df = present_to_predict[present_to_predict['user_id']==user_id_to_pred]
    output_dict[user_id_to_pred] = run_predict(temp_df)